In [1]:
##Importing required libraries 

import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [2]:
##Using Sequential API from Keras to build a sequential model

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(7, activation='softmax'))
    return model

In [3]:
##Capturing emotions using OpenCV (cv2)

def capture_emotion(model):
    cap = cv2.VideoCapture(0)
    emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

    while True:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            face = gray[y:y+h, x:x+w]
            face = cv2.resize(face, (48, 48))
            face = face / 255.0
            face = np.expand_dims(face, axis=0)
            face = np.expand_dims(face, axis=-1)

            prediction = model.predict(face)
            emotion = emotions[np.argmax(prediction)]

            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        cv2.imshow('Emotion Recognition', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [4]:
##Loading Training and Testing Data

def load_data(data_dir):
    emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    data = []
    labels = []

    for i, emotion in enumerate(emotions):
        emotion_dir = os.path.join(data_dir, emotion)
        for image_name in os.listdir(emotion_dir):
            image_path = os.path.join(emotion_dir, image_name)
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read the image in grayscale
            img = cv2.resize(img, (48, 48))  # Resize the image to the desired shape
            img = img / 255.0  # Normalize the pixel values to the range [0, 1]
            data.append(img)
            labels.append(i)

    data = np.array(data)
    data = np.expand_dims(data, axis=-1)
    labels = np.array(labels)

    return data, labels

In [8]:
## Loading the data
train_data, train_labels = load_data('training')
test_data, test_labels = load_data('test')

In [9]:
from keras.utils import to_categorical
from keras.optimizers import Adam

model = create_model()
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
##Training the model

model.fit(train_data, train_labels, batch_size=64, epochs=30, validation_data=(test_data, test_labels))

Epoch 1/30
113/113 [==============================] - 46s 387ms/step - loss: 1.8504 - accuracy: 0.2290 - val_loss: 1.8051 - val_accuracy: 0.2498
Epoch 2/30
113/113 [==============================] - 43s 378ms/step - loss: 1.7933 - accuracy: 0.2626 - val_loss: 1.7251 - val_accuracy: 0.3240
Epoch 3/30
113/113 [==============================] - 32s 281ms/step - loss: 1.7236 - accuracy: 0.3136 - val_loss: 1.6652 - val_accuracy: 0.3277
Epoch 4/30
113/113 [==============================] - 30s 267ms/step - loss: 1.6713 - accuracy: 0.3338 - val_loss: 1.6059 - val_accuracy: 0.3904
Epoch 5/30
113/113 [==============================] - 27s 243ms/step - loss: 1.6206 - accuracy: 0.3654 - val_loss: 1.5598 - val_accuracy: 0.4164
Epoch 6/30
113/113 [==============================] - 27s 243ms/step - loss: 1.5891 - accuracy: 0.3782 - val_loss: 1.5143 - val_accuracy: 0.4358
Epoch 7/30
113/113 [==============================] - 27s 243ms/step - loss: 1.5721 - accuracy: 0.3809 - val_loss: 1.4866 - val_ac

In [11]:
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

225/225 [==============================] - 15s 67ms/step - loss: 0.6086 - accuracy: 0.7903
Test accuracy: 0.7903315424919128


In [12]:
##Lunching Real-Time Emotion Detection

capture_emotion(model)

1/1 [==============================] - 0s 35ms/step
